# First simulation

A (Monte Carlo) Simulation consists in (1) [options](#setting-up-simulation-options), defining all the necessary parameters to setup the simulation, and (2) [results](#results), containing all the outputs of a simulation. One or more simulations form a `Project`. A **pyMonteCarlo** project stored on disk has the extension `.mcsim`. It consists of a [HDF5](http://hdf5group.org) file and can be opened in the [HDFViewer](http://hdf5group.org) or using any HDF5 library.

## Setting up simulation options

The options are defined by the class `Options`. It contains all the parameters necessary to run a simulation. The parameters are grouped into four categories: 

* program
* beam
* sample
* analyses

The beam, sample and analyses are independent of Monte Carlo programs. In other words, the same sample definition can be used for different Monte Carlo programs. For a given `Options` instance, only the program needs to change to run the same simulation with different Monte Carlo programs. That being said not all beam, sample and analyses are supported by all Monte Carlo programs. Supported parameters for each Monte Carlo program are listed in the [supported options](../supported-options.rst) page.

### Program

The program is specific to a particular Monte Carlo program. Each program follows the contract specified by the base program class `Program`. One implementation is for [Casino 2](http://www.gel.usherbrooke.ca/casino) as part of the package **pymontecarlo-casino2**. The program can be imported as follow:

In [ ]:
from pymontecarlo_casino2.program import Casino2Program

The parameters associated with the program will depend on each Monte Carlo program. For Casino 2, the number of trajectories and the models used for the simulation can be specified. Here is an example with the default models and 5000 trajectories:

In [ ]:
program = Casino2Program(5000)

Throughout **pyMonteCarlo**, a parameter can also be set/modified using its attribute inside the class:

In [ ]:
program.number_trajectories = 6000

All parameters are completely mutable and are only validated before a simulation starts.

### Beam

The second category of parameters is the beam. At the moment, three types of beam are implemented/supported: 

* a pencil beam: beam with no diameter, 
* a Gaussian beam: beam where the electrons are randomly distributed following a two dimensional Gaussian distribution, where the diameter is defined as full width at half maximum (FWHM),
* a cylindrical beam: beam where the electrons are randomly distributed within a cylinder.

All beam implementations must define the energy and type of the incident particles as defined by the base `Beam` class. The type of incident particle is defined for future expansions, since all  currently supported Monte Carlo programs only accept `ELECTRON`. Unless otherwise stated, all beams assume that the incident particles travel downwards along the z-axis, i.e. following the vector `(0, 0, -1)`.

The pencil beam is the most supported by the different Monte Carlo programs as no diameter is defined. Here is an example of a pencil beam with a beam energy of 15keV:

In [ ]:
from pymontecarlo.options.beam import PencilBeam
beam = PencilBeam(15e3)

Other parameters of the beam are the beam center position. The actual position of each particle will be randomly sampled by the Monte Carlo program to obtain a two dimensional Gaussian distribution centered at the specified position. By default, the beam is centered at `x = 0m` and `y = 0m`. The position can be changed using either attribute:

In [ ]:
beam.x_m = 100e-9
beam.y_m = 200e-9

If you are looking to perform a line scan, you should have a look at the pencil beam builder class `PencilBeamBuilder`. Builder classes are helper classes to create multiple instances by varying one or more parameters. For example, if we would like to create a pencil beam at two incident energy (5 and 15kV) and scan the sample from -100 to 100μm with a step size of 25μm:

In [ ]:
from pymontecarlo.options.beam import PencilBeamBuilder
builder = PencilBeamBuilder()
builder.add_energy_eV(5e3)
builder.add_energy_keV(15)
builder.add_linescan_x(-100e-6, 100e-6, 25e-6)
beams = builder.build()

beams

The variable `beams` is a `list` of 16 pencil beams, 2 incident electron energies and 8 positions in the linescan. Note however that each `Options` instance can only take one beam. This will result in an *exception* at validation.

### Sample

The sample parameter defines the geometry and the materials of the sample 
being bombarded by the incident particles.
There are currently 5 types of sample implemented:

* substrate (`SubstrateSample`): An infinitely thick sample. 
* inclusion (`InclusionSample`): An half-sphere inclusion in a substrate.
* horizontal layered (`HorizontalLayerSample`): Creates a multi-layers geometry. 
  The layers are assumed to be in the x-y plane (normal parallel to z) at tilt of 0.0°.
* vertical layered (`VericalLayerSample`): Creates a grain boundaries sample.
  It consists of 0 or many layers in the y-z plane (normal parallel to x) simulating interfaces between different materials.
  If no layer is defined, the geometry is a couple.
* sphere (`SphereSample`): A sphere in vacuum.
    
For all types of sample, the sample is entirely located below the ``z = 0`` plane.
While some Monte Carlo programs support custom and complex sample definitions, it was chosen for simplicity and compatibility to constrain the available types of sample.
If you would like to suggest/contribute another type of sample, please open an enhancement [issue](https://github.com/pymontecarlo/pymontecarlo/issues) or submit a [pull request](https://github.com/pymontecarlo/pymontecarlo/pulls).


Before creating a sample, material(s) must be defined.
A material defines the composition and density in a part of the sample (e.g. layer or substrate).
After importing the `Material` class, 

In [ ]:
from pymontecarlo.options.material import Material

There are three ways to create a material:

1. Pure, single element material:

In [ ]:
material = Material.pure(14) # pure silicon

2. A chemical formula:

In [ ]:
material = Material.from_formula('SiO2')

3. Composition in mass fraction. 
   The composition is expressed as a :class:`dict` where keys are atomic numbers and values, mass fractions:

In [ ]:
composition = {29: 0.4, 30: 0.6}
material = Material('Brass', composition)

In all three cases the mass density (in kg/m3) can be specified as an argument or set from its attribute:

In [ ]:
material.density_kg_per_m3 = 8400
material.density_g_per_cm3 = 8.4

If the density is not specified, it is calculated using this following formula:

$$\frac{1}{\rho} = \sum{\frac{m_i}{\rho_i}}$$

where $\rho_i$ and $m_i$ are respectively the elemental mass density and mass fraction of element $i$.

Each sample has different methods and variables to setup the materials. 
Here is an example for the substrate sample:

In [ ]:
from pymontecarlo.options.sample import SubstrateSample
from pymontecarlo.options.material import Material

copper = Material.pure(29)
substrate = SubstrateSample(copper)

and here is an example for the horizontal layered sample. 
The substrate is set to copper and two layers are added on top, forming from top to bottom: 100nm of SiO2, 50nm of brass and then copper:

In [ ]:
from pymontecarlo.options.sample import HorizontalLayerSample
from pymontecarlo.options.material import Material

copper = Material.pure(29)
sio2 = Material.from_formula('SiO2')
brass = Material('Brass', {29: 0.4, 30: 0.6})

sample = HorizontalLayerSample(copper)
sample.add_layer(sio2, 100e-9)
sample.add_layer(brass, 50e-9)

One trick to make sure the sample is properly setup is to draw it.
**pyMonteCarlo** uses [matplotlib](http://matplotlib.org) to draw the sample in 2D along the XZ, YZ or XY perspective.
Here is an example:

In [ ]:
import matplotlib.pyplot as plt
from pymontecarlo.figures.sample import SampleFigure, Perspective

plt.figure()
plt.subplot("111")

samplefig = SampleFigure(sample)
samplefig.perspective = Perspective.XZ
samplefig.draw(plt.gca())

plt.show()

### Analyses

## Running simulation(s)


## Interpreting simulation results
